# 🔐 URL Security Classifier — Model Training

## Ensemble: XGBoost + DistilBERT (Binary Classification)

**Goal**: Train a high-accuracy model to classify URLs as **Safe** or **Malicious**

**Architecture**:
1. **XGBoost** — 100+ hand-crafted URL features → explainable, fast inference
2. **DistilBERT** — Fine-tuned on raw URL text → learns character-level patterns
3. **Ensemble** — Weighted combination of calibrated probabilities → best of both

**Dataset**: [Kaggle Malicious URLs](https://www.kaggle.com/datasets/sid321axn/malicious-urls-dataset) (~651K URLs → binary)

**Output**: Packaged models ready for server integration

In [1]:
# ============================================================
# Install Dependencies (run this cell first!)
# ============================================================
!pip install -q kagglehub xgboost optuna scikit-learn transformers accelerate matplotlib seaborn tqdm

import warnings
warnings.filterwarnings('ignore')
print("✅ All packages installed!")

---
## 📊 Section 1: Data Loading & Preprocessing
Download the dataset, explore class distribution, convert to binary (Safe vs Malicious), and split into train/val/test.

In [2]:
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# ── 1.1 Download Dataset ─────────────────────────────────────
import kagglehub

print("📥 Downloading dataset from Kaggle...")
path = kagglehub.dataset_download("sid321axn/malicious-urls-dataset")
print(f"   Downloaded to: {path}")

csv_files = glob.glob(os.path.join(path, "**", "*.csv"), recursive=True)
csv_file = csv_files[0] if csv_files else os.path.join(path, "malicious_phish.csv")

df = pd.read_csv(csv_file)
print(f"\n✅ Loaded {df.shape[0]:,} URLs  |  Columns: {list(df.columns)}")

# ── 1.2 Explore ──────────────────────────────────────────────
print("\n📊 Original class distribution:")
print(df['type'].value_counts().to_string())

# ── 1.3 Convert to Binary ────────────────────────────────────
df['label'] = (df['type'] != 'benign').astype(int)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
df['type'].value_counts().plot.bar(ax=axes[0], color=['#34C759','#FF3B30','#FF9500','#007AFF'])
axes[0].set_title('Original 4-Class Distribution'); axes[0].set_ylabel('Count')
df['label'].value_counts().rename({0:'Safe', 1:'Malicious'}).plot.bar(ax=axes[1], color=['#34C759','#FF3B30'])
axes[1].set_title('Binary (Safe vs Malicious)'); axes[1].set_ylabel('Count')
plt.tight_layout(); plt.show()

# ── 1.4 Clean ────────────────────────────────────────────────
before = len(df)
df = df.dropna(subset=['url','label']).drop_duplicates(subset=['url'])
print(f"\n🧹 Removed {before - len(df):,} rows → {len(df):,} URLs remaining")

# ── 1.5 Split: 70 / 15 / 15 ─────────────────────────────────
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label'])
val_df, test_df   = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

print(f"\n📊 Split:")
print(f"   Train:      {len(train_df):>8,}")
print(f"   Validation: {len(val_df):>8,}")
print(f"   Test:       {len(test_df):>8,}")

---
## 🌲 Section 2: XGBoost with Feature Engineering

Hand-crafted **100+ features** covering:
- **Length** — URL, domain, path, query, subdomain, TLD lengths
- **Counts** — dots, hyphens, digits, special chars, subdomains, path depth
- **Ratios** — digit/letter/special proportions, domain-to-URL ratio
- **Entropy** — Shannon entropy of domain, path, query, subdomain (randomness detection)
- **Boolean** — IP address, port, HTTPS, hex encoding, punycode, @ symbol
- **TLD** — suspicious/trusted TLD classification
- **Keywords** — phishing, malware, brand impersonation detection
- **Structure** — deep paths, embedded URLs, base64 patterns, shorteners

In [3]:
import re, math
from urllib.parse import urlparse, parse_qs
from collections import Counter

# ── Keyword / Pattern Dictionaries ────────────────────────────

SUSPICIOUS_TLDS = frozenset({
    "tk","ml","ga","cf","gq","pw","top","xyz","club","work","click","link",
    "surf","buzz","fun","monster","quest","cam","icu","cc","ws","info","biz",
    "su","ru","cn","online","site","website","space","tech","store","stream",
    "download","win","review","racing","cricket","science","party","gdn",
    "loan","men","country","kim","date","faith","accountant","bid","trade","webcam",
})
TRUSTED_TLDS = frozenset({
    "edu","gov","mil","int","ac.uk","gov.uk","edu.au","gov.au",
})
BRAND_KEYWORDS = frozenset({
    "paypal","apple","google","microsoft","amazon","facebook","netflix",
    "instagram","whatsapp","twitter","linkedin","ebay","dropbox","icloud",
    "outlook","office365","yahoo","chase","wellsfargo","bankofamerica",
    "citibank","capitalone","steam","spotify","adobe","coinbase","binance","metamask",
})
PHISHING_KEYWORDS = frozenset({
    "login","signin","sign-in","logon","password","verify","verification",
    "confirm","update","secure","security","account","banking","wallet",
    "suspend","suspended","urgent","expire","unlock","restore","recover",
    "validate","authenticate","webscr","customer","support","helpdesk",
})
MALWARE_KEYWORDS = frozenset({
    "download","free","crack","keygen","patch","serial","warez","torrent",
    "nulled","hack","cheat","generator","install","setup","update","flash",
    "player","codec","driver",
})
URL_SHORTENERS = frozenset({
    "bit.ly","goo.gl","tinyurl.com","ow.ly","t.co","is.gd",
    "buff.ly","adf.ly","j.mp","rb.gy","cutt.ly","tiny.cc",
})
DANGEROUS_EXTS = frozenset({
    ".exe",".dll",".bat",".cmd",".msi",".scr",".pif",".vbs",
    ".js",".jar",".apk",".dmg",".zip",".rar",".7z",".iso",
})

# ── Helper Functions ──────────────────────────────────────────

def calc_entropy(text: str) -> float:
    """Shannon entropy — higher = more random."""
    if not text: return 0.0
    freq = Counter(text.lower())
    n = len(text)
    return -sum((c/n) * math.log2(c/n) for c in freq.values() if c > 0)

def max_run(text: str, cond) -> int:
    """Longest consecutive run of chars matching cond."""
    best = cur = 0
    for ch in text:
        if cond(ch): cur += 1; best = max(best, cur)
        else: cur = 0
    return best

# ── Main Feature Extractor ────────────────────────────────────

def extract_features(url: str) -> dict:
    """Extract 100+ features from a single URL."""
    f = {}
    url = str(url).strip()

    try:
        parsed = urlparse(url if "://" in url else f"http://{url}")
    except Exception:
        return {k: 0 for k in FEATURE_NAMES}

    scheme   = parsed.scheme.lower()
    netloc   = parsed.netloc.lower()
    path     = parsed.path
    query    = parsed.query
    fragment = parsed.fragment

    # Domain cleanup
    netloc_no_port = netloc.split(":")[0] if (":" in netloc and not netloc.startswith("[")) else netloc
    domain = netloc_no_port
    parts  = domain.split(".")
    path_parts = [p for p in path.split("/") if p]
    subdomain  = ".".join(parts[:-2]) if len(parts) > 2 else ""
    tld = parts[-1] if parts else ""
    if len(parts) >= 3 and parts[-2] in ("co","com","org","net","gov","ac","edu"):
        tld = f"{parts[-2]}.{parts[-1]}"

    url_lower  = url.lower()
    path_lower = path.lower()

    # ═══ LENGTH ═══
    f['url_length']         = len(url)
    f['domain_length']      = len(domain)
    f['path_length']        = len(path)
    f['query_length']       = len(query)
    f['fragment_length']    = len(fragment)
    f['subdomain_length']   = len(subdomain)
    f['tld_length']         = len(tld)
    f['longest_domain_part']= max((len(p) for p in parts), default=0)
    f['avg_domain_part_len']= np.mean([len(p) for p in parts]) if parts else 0
    f['longest_path_part']  = max((len(p) for p in path_parts), default=0)
    f['avg_path_part_len']  = np.mean([len(p) for p in path_parts]) if path_parts else 0

    # ═══ COUNTS ═══
    for ch, name in [(".",  "dot"),  ("-", "hyphen"), ("_", "underscore"),
                     ("/",  "slash"),("?", "question"),("=","equals"),
                     ("&",  "amp"),  ("@", "at"),     ("%", "percent"),
                     ("~",  "tilde"),("#", "hash"),   (":", "colon"),
                     (";",  "semicolon")]:
        f[f'{name}_count'] = url.count(ch)

    f['domain_dot_count']    = domain.count(".")
    f['domain_hyphen_count'] = domain.count("-")
    f['domain_digit_count']  = sum(c.isdigit() for c in domain)
    f['subdomain_count']     = max(0, len(parts) - 2)
    f['path_depth']          = len(path_parts)
    f['digit_count']         = sum(c.isdigit() for c in url)
    f['letter_count']        = sum(c.isalpha() for c in url)
    f['uppercase_count']     = sum(c.isupper() for c in url)
    f['special_char_count']  = sum(not c.isalnum() for c in url)

    try:
        qp = parse_qs(query)
        f['query_param_count']     = len(qp)
        f['query_value_total_len'] = sum(len(v) for vals in qp.values() for v in vals)
    except Exception:
        f['query_param_count'] = 0; f['query_value_total_len'] = 0

    # ═══ RATIOS ═══
    ul = max(len(url), 1); dl = max(len(domain), 1)
    f['digit_ratio']         = f['digit_count'] / ul
    f['letter_ratio']        = f['letter_count'] / ul
    f['special_char_ratio']  = f['special_char_count'] / ul
    f['uppercase_ratio']     = f['uppercase_count'] / max(f['letter_count'], 1)
    f['domain_digit_ratio']  = f['domain_digit_count'] / dl
    f['domain_hyphen_ratio'] = f['domain_hyphen_count'] / dl
    f['path_url_ratio']      = f['path_length'] / ul
    f['query_url_ratio']     = f['query_length'] / ul
    f['domain_url_ratio']    = f['domain_length'] / ul

    # ═══ ENTROPY ═══
    f['url_entropy']       = calc_entropy(url)
    f['domain_entropy']    = calc_entropy(domain.replace(".", ""))
    f['path_entropy']      = calc_entropy(path)
    f['query_entropy']     = calc_entropy(query)
    f['subdomain_entropy'] = calc_entropy(subdomain)

    # ═══ BOOLEAN ═══
    f['is_https']                = int(scheme == "https")
    f['is_http']                 = int(scheme == "http")
    f['has_www']                 = int(domain.startswith("www."))
    f['has_port']                = int(":" in netloc and not netloc.startswith("["))
    f['has_at_symbol']           = int("@" in url)
    f['has_double_slash_in_path']= int("//" in path)
    f['has_hex_encoding']        = int(bool(re.search(r"%[0-9a-fA-F]{2}", url)))
    f['has_punycode']            = int("xn--" in domain)
    f['has_ip_address']          = int(bool(re.match(r"^(\d{1,3}\.){3}\d{1,3}$", domain)))
    f['has_hex_ip']              = int(bool(re.match(r"^(0x[0-9a-f]+\.){3}0x[0-9a-f]+$", domain)))
    f['has_ip_like']             = int(domain.replace(".", "").isdigit() and len(domain) > 6)

    # ═══ TLD ═══
    f['is_suspicious_tld'] = int(tld in SUSPICIOUS_TLDS)
    f['is_trusted_tld']    = int(tld in TRUSTED_TLDS)
    f['is_com']            = int(tld == "com")
    f['is_org']            = int(tld == "org")
    f['is_net']            = int(tld == "net")
    f['is_country_tld']    = int(len(tld) == 2 and tld.isalpha())

    # ═══ CHARACTER DISTRIBUTION ═══
    f['max_consec_digits']  = max_run(url, str.isdigit)
    f['max_consec_letters'] = max_run(url, str.isalpha)
    f['max_consec_special'] = max_run(url, lambda c: not c.isalnum())
    vowels = set("aeiou")
    dom_letters = [c for c in domain if c.isalpha()]
    f['domain_vowel_ratio'] = sum(c in vowels for c in dom_letters) / max(len(dom_letters), 1)

    # ═══ KEYWORDS ═══
    f['brand_keyword_count']    = sum(1 for b in BRAND_KEYWORDS if b in url_lower)
    f['has_brand_in_subdomain'] = int(any(b in subdomain.lower() for b in BRAND_KEYWORDS))
    f['phishing_keyword_count'] = sum(1 for k in PHISHING_KEYWORDS if k in url_lower)
    f['malware_keyword_count']  = sum(1 for k in MALWARE_KEYWORDS if k in url_lower)
    f['is_url_shortener']       = int(any(s in netloc for s in URL_SHORTENERS))
    f['has_dangerous_ext']      = int(any(path_lower.endswith(e) for e in DANGEROUS_EXTS))
    f['has_exe']                = int(path_lower.endswith(".exe"))
    f['has_php']                = int(".php" in path_lower)

    # ═══ STRUCTURAL PATTERNS ═══
    f['has_double_letters']  = int(bool(re.search(r"(.)\1", domain)))
    f['has_long_subdomain']  = int(len(subdomain) > 20)
    f['has_deep_path']       = int(len(path_parts) > 5)
    f['has_embedded_url']    = int("http" in path_lower or "www" in path_lower)
    f['has_data_uri']        = int(url_lower.startswith("data:"))
    f['has_javascript']      = int("javascript:" in url_lower)
    f['has_base64']          = int(bool(re.search(r"[A-Za-z0-9+/]{20,}={0,2}", url)))
    f['brand_in_domain']     = int(any(b in domain for b in BRAND_KEYWORDS))
    f['brand_not_registered']= int(
        f['brand_in_domain'] == 1 and not any(
            domain == f"{b}.com" or domain == f"www.{b}.com" for b in BRAND_KEYWORDS
        )
    )

    return f

# Build canonical feature name list
FEATURE_NAMES = list(extract_features("https://www.example.com/path?q=1").keys())
print(f"✅ Feature engineering ready — {len(FEATURE_NAMES)} features per URL")

In [4]:
from tqdm import tqdm

# ── 2.1 Extract Features from All URLs ───────────────────────
print("⚙️  Extracting features (this takes a few minutes)...\n")

def extract_batch(urls, desc="Extracting"):
    return pd.DataFrame(
        [extract_features(str(u)) for u in tqdm(urls, desc=desc)],
        columns=FEATURE_NAMES
    ).fillna(0).astype(np.float32)

X_train_feat = extract_batch(train_df['url'].tolist(), "  Train")
X_val_feat   = extract_batch(val_df['url'].tolist(),   "  Val")
X_test_feat  = extract_batch(test_df['url'].tolist(),  "  Test")

y_train = train_df['label'].values
y_val   = val_df['label'].values
y_test  = test_df['label'].values

# Class imbalance ratio (used by both XGBoost and DistilBERT later)
scale_pos = float((y_train == 0).sum() / (y_train == 1).sum())

print(f"\n✅ Feature matrices ready:")
print(f"   Train: {X_train_feat.shape}")
print(f"   Val:   {X_val_feat.shape}")
print(f"   Test:  {X_test_feat.shape}")
print(f"   Class ratio (safe/malicious): {scale_pos:.2f}")

In [5]:
import xgboost as xgb
import optuna
import torch
from sklearn.metrics import f1_score
from sklearn.calibration import CalibratedClassifierCV

# ── 2.2 Hyperparameter Tuning with Optuna (50 trials) ────────
print("🔍 Running Optuna hyperparameter search...\n")

USE_GPU = torch.cuda.is_available()
print(f"   GPU available: {USE_GPU}")

def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'device': 'cuda' if USE_GPU else 'cpu',
        'tree_method': 'hist',
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': 1500,
        'early_stopping_rounds': 50,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'gamma': trial.suggest_float('gamma', 0, 5.0),
        'scale_pos_weight': scale_pos,
        'random_state': 42,
        'verbosity': 0,
    }
    m = xgb.XGBClassifier(**params)
    m.fit(X_train_feat.values, y_train,
          eval_set=[(X_val_feat.values, y_val)], verbose=False)
    return f1_score(y_val, m.predict(X_val_feat.values))

optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

print(f"\n🏆 Best trial F1: {study.best_value:.4f}")
print(f"   Best params:")
for k, v in study.best_params.items():
    print(f"     {k}: {v}")

# ── 2.3 Train Final XGBoost with Best Params ─────────────────
bp = study.best_params.copy()
bp.update({
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'device': 'cuda' if USE_GPU else 'cpu',
    'tree_method': 'hist',
    'n_estimators': 3000,          # High ceiling — early stopping picks best
    'early_stopping_rounds': 100,  # More patience for final model
    'scale_pos_weight': scale_pos,
    'random_state': 42,
    'verbosity': 1,
})

xgb_model = xgb.XGBClassifier(**bp)
xgb_model.fit(X_train_feat.values, y_train,
              eval_set=[(X_val_feat.values, y_val)], verbose=True)

print(f"\n   Best iteration: {xgb_model.best_iteration}")

# ── 2.4 Calibrate Probabilities (Platt Scaling) ──────────────
xgb_calibrated = CalibratedClassifierCV(xgb_model, method='sigmoid', cv='prefit')
xgb_calibrated.fit(X_val_feat.values, y_val)

print("✅ XGBoost trained & calibrated!")

In [6]:
from sklearn.metrics import (
    classification_report, confusion_matrix,
    roc_curve, auc, roc_auc_score,
    precision_recall_curve, average_precision_score, accuracy_score
)

# ── 2.5 Evaluate XGBoost on Test Set ─────────────────────────
xgb_probs = xgb_calibrated.predict_proba(X_test_feat.values)[:, 1]
xgb_preds = (xgb_probs >= 0.5).astype(int)

print("=" * 55)
print("📊 XGBoost — Test Set Results")
print("=" * 55)
print(classification_report(y_test, xgb_preds, target_names=['Safe', 'Malicious']))
print(f"ROC-AUC: {roc_auc_score(y_test, xgb_probs):.4f}")

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, xgb_preds), annot=True, fmt='d', cmap='Blues',
            xticklabels=['Safe','Malicious'], yticklabels=['Safe','Malicious'], ax=axes[0])
axes[0].set_title('XGBoost — Confusion Matrix')
axes[0].set_xlabel('Predicted'); axes[0].set_ylabel('Actual')

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, xgb_probs)
axes[1].plot(fpr, tpr, 'b-', linewidth=2, label=f'AUC = {auc(fpr, tpr):.4f}')
axes[1].plot([0, 1], [0, 1], 'k--', alpha=0.5)
axes[1].set_title('ROC Curve'); axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate'); axes[1].legend()

# Feature Importance (Top 20)
imp = xgb_model.feature_importances_
top20 = np.argsort(imp)[-20:]
axes[2].barh(range(20), imp[top20], color='steelblue')
axes[2].set_yticks(range(20))
axes[2].set_yticklabels([FEATURE_NAMES[i] for i in top20])
axes[2].set_title('Top 20 Feature Importances')

plt.tight_layout(); plt.show()

---
## 🤖 Section 3: DistilBERT Fine-Tuning

Fine-tune `distilbert-base-uncased` directly on **raw URL text**.
The transformer learns character-level and subword-level patterns that hand-crafted features miss (e.g., subtle typosquatting, obfuscated paths).

- **3 epochs** with linear warmup + learning rate decay
- **Mixed precision** (FP16) for faster training on GPU
- **Class-weighted loss** to handle imbalance

In [7]:
from torch.utils.data import Dataset, DataLoader
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    get_linear_schedule_with_warmup,
)

# ── 3.1 Dataset & DataLoaders ────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  Device: {device}")
if device.type == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

bert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

class URLDataset(Dataset):
    def __init__(self, urls, labels, tokenizer, max_len=128):
        self.urls = urls
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.urls)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            str(self.urls[idx]),
            truncation=True, padding='max_length',
            max_length=self.max_len, return_tensors='pt'
        )
        return {
            'input_ids':      enc['input_ids'].squeeze(),
            'attention_mask': enc['attention_mask'].squeeze(),
            'label':          torch.tensor(self.labels[idx], dtype=torch.long),
        }

BS = 64
train_ds = URLDataset(train_df['url'].tolist(), train_df['label'].tolist(), bert_tokenizer)
val_ds   = URLDataset(val_df['url'].tolist(),   val_df['label'].tolist(),   bert_tokenizer)
test_ds  = URLDataset(test_df['url'].tolist(),  test_df['label'].tolist(),  bert_tokenizer)

train_loader = DataLoader(train_ds, batch_size=BS, shuffle=True,  num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BS, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BS, shuffle=False, num_workers=2, pin_memory=True)

print(f"\n✅ DataLoaders ready:")
print(f"   Train: {len(train_loader)} batches")
print(f"   Val:   {len(val_loader)} batches")
print(f"   Test:  {len(test_loader)} batches")

In [8]:
# ── 3.2 Fine-Tune DistilBERT ──────────────────────────────────
EPOCHS = 3
LR = 2e-5

bert_model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
).to(device)

optimizer = torch.optim.AdamW(bert_model.parameters(), lr=LR, weight_decay=0.01)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(total_steps * 0.1),
    num_training_steps=total_steps,
)
scaler = torch.cuda.amp.GradScaler(enabled=(device.type == 'cuda'))

# Class-weighted loss for imbalance
weights = torch.tensor([1.0, scale_pos], dtype=torch.float).to(device)
loss_fn = torch.nn.CrossEntropyLoss(weight=weights)

best_f1 = 0
history = {'train_loss': [], 'val_loss': [], 'val_f1': []}

for epoch in range(EPOCHS):
    # ── Train ──
    bert_model.train()
    t_loss, t_steps = 0.0, 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]")
    for batch in pbar:
        ids    = batch['input_ids'].to(device)
        mask   = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
            out  = bert_model(input_ids=ids, attention_mask=mask)
            loss = loss_fn(out.logits, labels)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(bert_model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        t_loss += loss.item(); t_steps += 1
        pbar.set_postfix(loss=f"{t_loss/t_steps:.4f}")

    # ── Validate ──
    bert_model.eval()
    v_loss, v_steps = 0.0, 0
    preds_all, labels_all = [], []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]"):
            ids    = batch['input_ids'].to(device)
            mask   = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
                out  = bert_model(input_ids=ids, attention_mask=mask)
                loss = loss_fn(out.logits, labels)

            v_loss += loss.item(); v_steps += 1
            preds_all.extend(torch.argmax(out.logits, 1).cpu().numpy())
            labels_all.extend(labels.cpu().numpy())

    vf1 = f1_score(labels_all, preds_all)
    history['train_loss'].append(t_loss / t_steps)
    history['val_loss'].append(v_loss / v_steps)
    history['val_f1'].append(vf1)

    print(f"\n  → train_loss={t_loss/t_steps:.4f}  val_loss={v_loss/v_steps:.4f}  val_f1={vf1:.4f}")

    if vf1 > best_f1:
        best_f1 = vf1
        torch.save(bert_model.state_dict(), "best_distilbert.pt")
        print(f"  💾 Saved best model (F1={vf1:.4f})")

# Reload best checkpoint
bert_model.load_state_dict(torch.load("best_distilbert.pt"))
print(f"\n✅ DistilBERT training complete! Best Val F1: {best_f1:.4f}")

In [9]:
# ── 3.3 Evaluate DistilBERT on Test Set ───────────────────────
bert_model.eval()
all_probs = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="DistilBERT test"):
        ids  = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
            out = bert_model(input_ids=ids, attention_mask=mask)
        all_probs.extend(torch.softmax(out.logits, dim=1)[:, 1].cpu().numpy())

bert_probs = np.array(all_probs)
bert_preds = (bert_probs >= 0.5).astype(int)

print("=" * 55)
print("📊 DistilBERT — Test Set Results")
print("=" * 55)
print(classification_report(y_test, bert_preds, target_names=['Safe', 'Malicious']))
print(f"ROC-AUC: {roc_auc_score(y_test, bert_probs):.4f}")

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, bert_preds), annot=True, fmt='d', cmap='Oranges',
            xticklabels=['Safe','Malicious'], yticklabels=['Safe','Malicious'], ax=axes[0])
axes[0].set_title('DistilBERT — Confusion Matrix')
axes[0].set_xlabel('Predicted'); axes[0].set_ylabel('Actual')

# Training Curves
ax1 = axes[1]
ax1.plot(range(1, EPOCHS+1), history['train_loss'], 'b-o', label='Train Loss')
ax1.plot(range(1, EPOCHS+1), history['val_loss'],   'r-o', label='Val Loss')
ax1.set_xlabel('Epoch'); ax1.set_ylabel('Loss'); ax1.set_title('Training Curves')
ax1.legend(loc='upper left')

ax2 = ax1.twinx()
ax2.plot(range(1, EPOCHS+1), history['val_f1'], 'g--s', label='Val F1')
ax2.set_ylabel('F1 Score'); ax2.legend(loc='upper right')

plt.tight_layout(); plt.show()

---
## 🏆 Section 4: Ensemble — Combining Both Models

Weighted average of calibrated probabilities:

$$P_{ensemble} = \alpha \cdot P_{XGBoost} + (1 - \alpha) \cdot P_{DistilBERT}$$

The optimal weight $\alpha$ is found by grid search on the **validation set**, then evaluated on the **test set**.

In [10]:
# ══════════════════════════════════════════════════════════════
# 4.1 Find Optimal Ensemble Weight (validation set)
# ══════════════════════════════════════════════════════════════

# XGBoost validation probabilities
xgb_val_p = xgb_calibrated.predict_proba(X_val_feat.values)[:, 1]

# DistilBERT validation probabilities
bert_model.eval()
bvp = []
with torch.no_grad():
    for batch in tqdm(val_loader, desc="BERT val probs"):
        ids  = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
            out = bert_model(input_ids=ids, attention_mask=mask)
        bvp.extend(torch.softmax(out.logits, dim=1)[:, 1].cpu().numpy())
bert_val_p = np.array(bvp)

# Grid search for optimal alpha
best_alpha, best_ens_f1 = 0.5, 0
for a in np.arange(0.0, 1.01, 0.05):
    p = ((a * xgb_val_p + (1 - a) * bert_val_p) >= 0.5).astype(int)
    f = f1_score(y_val, p)
    if f > best_ens_f1:
        best_alpha, best_ens_f1 = round(a, 2), f

print(f"🏆 Optimal α = {best_alpha:.2f}")
print(f"   XGBoost weight:    {best_alpha:.0%}")
print(f"   DistilBERT weight: {1-best_alpha:.0%}")
print(f"   Validation F1:     {best_ens_f1:.4f}")

# ══════════════════════════════════════════════════════════════
# 4.2 Final Test Evaluation — All 3 Models
# ══════════════════════════════════════════════════════════════
ens_probs = best_alpha * xgb_probs + (1 - best_alpha) * bert_probs
ens_preds = (ens_probs >= 0.5).astype(int)

print("\n" + "=" * 60)
print("🏆 ENSEMBLE — Final Test Set Results")
print("=" * 60)
print(classification_report(y_test, ens_preds, target_names=['Safe', 'Malicious']))

# ROC + PR curves comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for name, probs, c in [('XGBoost', xgb_probs, 'blue'),
                         ('DistilBERT', bert_probs, 'red'),
                         ('Ensemble', ens_probs, 'green')]:
    fpr, tpr, _ = roc_curve(y_test, probs)
    axes[0].plot(fpr, tpr, color=c, linewidth=2, label=f'{name} (AUC={auc(fpr, tpr):.4f})')
axes[0].plot([0, 1], [0, 1], 'k--', alpha=0.5)
axes[0].set_title('ROC Curves — Model Comparison')
axes[0].set_xlabel('False Positive Rate'); axes[0].set_ylabel('True Positive Rate')
axes[0].legend()

for name, probs, c in [('XGBoost', xgb_probs, 'blue'),
                         ('DistilBERT', bert_probs, 'red'),
                         ('Ensemble', ens_probs, 'green')]:
    pr, rc, _ = precision_recall_curve(y_test, probs)
    ap = average_precision_score(y_test, probs)
    axes[1].plot(rc, pr, color=c, linewidth=2, label=f'{name} (AP={ap:.4f})')
axes[1].set_title('Precision-Recall Curves')
axes[1].set_xlabel('Recall'); axes[1].set_ylabel('Precision')
axes[1].legend()

plt.tight_layout(); plt.show()

# Summary table
print("\n📊 Model Comparison Summary:")
print("─" * 55)
print(f"{'Model':<15} {'Accuracy':>10} {'F1':>8} {'ROC-AUC':>10}")
print("─" * 55)
for name, probs in [('XGBoost', xgb_probs), ('DistilBERT', bert_probs), ('Ensemble', ens_probs)]:
    p = (probs >= 0.5).astype(int)
    print(f"{name:<15} {accuracy_score(y_test, p):>10.4f} {f1_score(y_test, p):>8.4f} {roc_auc_score(y_test, probs):>10.4f}")
print("─" * 55)

---
## 💾 Section 5: Export Models for Server Integration

Saves everything needed to run inference on the FastAPI server:
- `xgb_model.pkl` — Calibrated XGBoost model
- `feature_names.json` — Feature name list (preserves column order)
- `distilbert_url_classifier/` — Fine-tuned DistilBERT (tokenizer + weights)
- `ensemble_config.json` — Ensemble weight α + performance metrics

In [11]:
import pickle, json

# ── 5.1 Save XGBoost ─────────────────────────────────────────
with open("xgb_model.pkl", "wb") as f:
    pickle.dump(xgb_calibrated, f)

with open("feature_names.json", "w") as f:
    json.dump(FEATURE_NAMES, f)

print(f"✅ XGBoost saved: xgb_model.pkl ({len(FEATURE_NAMES)} features)")

# ── 5.2 Save DistilBERT ──────────────────────────────────────
save_dir = "distilbert_url_classifier"
bert_model.save_pretrained(save_dir)
bert_tokenizer.save_pretrained(save_dir)
print(f"✅ DistilBERT saved: {save_dir}/")

# ── 5.3 Save Ensemble Config ─────────────────────────────────
config = {
    "xgb_weight": float(best_alpha),
    "bert_weight": float(1 - best_alpha),
    "threshold": 0.5,
    "n_features": len(FEATURE_NAMES),
    "xgb_model_file": "xgb_model.pkl",
    "feature_names_file": "feature_names.json",
    "bert_model_dir": save_dir,
    "metrics": {
        "xgb_accuracy":      float(accuracy_score(y_test, xgb_preds)),
        "xgb_f1":            float(f1_score(y_test, xgb_preds)),
        "xgb_auc":           float(roc_auc_score(y_test, xgb_probs)),
        "bert_accuracy":     float(accuracy_score(y_test, bert_preds)),
        "bert_f1":           float(f1_score(y_test, bert_preds)),
        "bert_auc":          float(roc_auc_score(y_test, bert_probs)),
        "ensemble_accuracy": float(accuracy_score(y_test, ens_preds)),
        "ensemble_f1":       float(f1_score(y_test, ens_preds)),
        "ensemble_auc":      float(roc_auc_score(y_test, ens_probs)),
    },
}

with open("ensemble_config.json", "w") as f:
    json.dump(config, f, indent=2)

print("✅ Ensemble config saved: ensemble_config.json")
print(json.dumps(config["metrics"], indent=2))

# ── 5.4 Package & Download ───────────────────────────────────
!zip -r url_classifier_models.zip \
    xgb_model.pkl \
    feature_names.json \
    distilbert_url_classifier/ \
    ensemble_config.json

print("\n📦 All models packaged: url_classifier_models.zip")

# Auto-download in Colab
try:
    from google.colab import files
    files.download("url_classifier_models.zip")
    print("⬇️  Download started!")
except ImportError:
    print("ℹ️  Not running in native Colab browser — download the zip manually from the file browser.")